In [1]:
import numpy as np
import os
import pandas as pd

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN, GRU
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

import warnings
warnings.filterwarnings(action='ignore')

#한글설정
import matplotlib.font_manager as fm

font_dirs = ['/usr/share/fonts/truetype/nanum', ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    fm.fontManager.addfont(font_file)
    
# 한글 출력을 위해서 폰트 옵션을 설정합니다.
# "axes.unicode_minus" : 마이너스가 깨질 것을 방지

sns.set(font="NanumBarunGothic",
        rc={"axes.unicode_minus":False},
        style='darkgrid')

#GPU 사용 설정, -1이면 CPU 사용
# os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# 시계열, LSTM

In [47]:
# 2. LSTM model
def lstm_model():
    seed_num = 42 # model_seed
    tf.random.set_seed(seed_num)

    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))
    return lstm

def sorted_ls(path):
    mtime=lambda f: os.stat(os.path.join(path, f)).st_mtime
    return list(sorted(os.listdir(path), key=mtime))

path = '/project/LSH/'
with tf.device('/device:GPU:0'):
    # 1. Data load
    x = np.load(path + 'x_(7727,10,4068).npy')
    y = np.load(path + 'y_(7727,1).npy')

    # 3-1. Best model saving
    MODEL_SAVE_FOLDER_PATH = './models/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    data={}
    sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)

    # 4. Crossvalidation
    for seed, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
        X_train, y_train = x[train_index,:,:], y[train_index]
        X_test, y_test = x[test_index,:,:], y[test_index]

        # 3-2. Best model saving
        model_path = MODEL_SAVE_FOLDER_PATH + f'/seed={seed}_'+'{epoch:02d}'+'_val_acc:{val_acc:.4f}.hdf5'

        cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc',
                                        verbose=1, save_best_only=True)
        early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=False)


        model = lstm_model()
        model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                      loss = "binary_crossentropy", metrics=['acc'])
        model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,
                  callbacks=[early_stop, cb_checkpoint], shuffle=False)

        # 5. accuracy checking 
#         from pathlib import Path
#         paths = sorted(Path(MODEL_SAVE_FOLDER_PATH).iterdir(), key=os.path.getmtime)[-1]
#         best_model_path = str(paths)
        best_model_path = MODEL_SAVE_FOLDER_PATH + sorted_ls('models/')[-1]
        print('############# model path : ', best_model_path)

        best_model = keras.models.load_model(best_model_path) 
#         y_pred_train = best_model.predict(X_train, batch_size=10000, workers=-1, use_multiprocessing=True)
        y_pred_test = best_model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)

        y_pred_test[y_pred_test>0.5]=1
        y_pred_test[y_pred_test<=0.5]=0

        acc = accuracy_score(y_test, y_pred_test)
        data[seed]=acc
        print(f'정확도 : {acc}, seed_num = {seed}')

    df = pd.DataFrame.from_dict(data, orient='index')
    print(f'정확도 df 만들고 평균 확인 : {df.mean().values}')    

0it [00:00, ?it/s]

Epoch 1/500
37/37 [==============================] - 6s 79ms/step - loss: 0.6753 - acc: 0.6004 - val_loss: 0.6704 - val_acc: 0.6054

Epoch 00001: val_acc improved from -inf to 0.60543, saving model to ./models/seed=0_01_val_acc:0.6054.hdf5
Epoch 2/500
37/37 [==============================] - 3s 71ms/step - loss: 0.6696 - acc: 0.6097 - val_loss: 0.6541 - val_acc: 0.6054

Epoch 00002: val_acc did not improve from 0.60543
Epoch 3/500
37/37 [==============================] - 3s 69ms/step - loss: 0.5884 - acc: 0.6785 - val_loss: 0.5440 - val_acc: 0.7296

Epoch 00003: val_acc improved from 0.60543 to 0.72962, saving model to ./models/seed=0_03_val_acc:0.7296.hdf5
Epoch 4/500
37/37 [==============================] - 3s 71ms/step - loss: 0.4601 - acc: 0.7970 - val_loss: 0.5489 - val_acc: 0.7393

Epoch 00004: val_acc improved from 0.72962 to 0.73933, saving model to ./models/seed=0_04_val_acc:0.7393.hdf5
Epoch 5/500
37/37 [==============================] - 3s 71ms/step - loss: 0.4108 - acc: 0.8

37/37 [==============================] - 3s 70ms/step - loss: 0.1294 - acc: 0.9577 - val_loss: 1.0653 - val_acc: 0.7180

Epoch 00041: val_acc did not improve from 0.74515
Epoch 42/500
37/37 [==============================] - 3s 70ms/step - loss: 0.1313 - acc: 0.9547 - val_loss: 1.0884 - val_acc: 0.6966

Epoch 00042: val_acc did not improve from 0.74515
Epoch 43/500
37/37 [==============================] - 3s 70ms/step - loss: 0.1265 - acc: 0.9594 - val_loss: 0.9660 - val_acc: 0.7316

Epoch 00043: val_acc did not improve from 0.74515
Epoch 44/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0688 - acc: 0.9847 - val_loss: 1.0030 - val_acc: 0.7277

Epoch 00044: val_acc did not improve from 0.74515
Epoch 45/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0608 - acc: 0.9853 - val_loss: 1.0478 - val_acc: 0.7257

Epoch 00045: val_acc did not improve from 0.74515
Epoch 46/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0613 - acc: 0.98


Epoch 00014: val_acc did not improve from 0.77684
Epoch 15/500
37/37 [==============================] - 3s 70ms/step - loss: 0.2279 - acc: 0.9184 - val_loss: 0.6949 - val_acc: 0.7309

Epoch 00015: val_acc did not improve from 0.77684
Epoch 16/500
37/37 [==============================] - 3s 69ms/step - loss: 0.2172 - acc: 0.9243 - val_loss: 0.6914 - val_acc: 0.7400

Epoch 00016: val_acc did not improve from 0.77684
Epoch 17/500
37/37 [==============================] - 3s 69ms/step - loss: 0.2327 - acc: 0.9118 - val_loss: 0.6651 - val_acc: 0.7555

Epoch 00017: val_acc did not improve from 0.77684
Epoch 18/500
37/37 [==============================] - 3s 69ms/step - loss: 0.2292 - acc: 0.9120 - val_loss: 0.6595 - val_acc: 0.7549

Epoch 00018: val_acc did not improve from 0.77684
Epoch 19/500
37/37 [==============================] - 3s 70ms/step - loss: 0.1655 - acc: 0.9465 - val_loss: 0.7198 - val_acc: 0.7471

Epoch 00019: val_acc did not improve from 0.77684
Epoch 20/500
37/37 [=========

정확도 : 0.7729624838292367, seed_num = 1
Epoch 1/500
37/37 [==============================] - 6s 88ms/step - loss: 0.6766 - acc: 0.6011 - val_loss: 0.6684 - val_acc: 0.6171

Epoch 00001: val_acc improved from -inf to 0.61708, saving model to ./models/seed=2_01_val_acc:0.6171.hdf5
Epoch 2/500
37/37 [==============================] - 3s 70ms/step - loss: 0.6718 - acc: 0.6011 - val_loss: 0.6474 - val_acc: 0.6171

Epoch 00002: val_acc did not improve from 0.61708
Epoch 3/500
37/37 [==============================] - 3s 69ms/step - loss: 0.5979 - acc: 0.6783 - val_loss: 0.5098 - val_acc: 0.7671

Epoch 00003: val_acc improved from 0.61708 to 0.76714, saving model to ./models/seed=2_03_val_acc:0.7671.hdf5
Epoch 4/500
37/37 [==============================] - 3s 69ms/step - loss: 0.4723 - acc: 0.7883 - val_loss: 0.4718 - val_acc: 0.7820

Epoch 00004: val_acc improved from 0.76714 to 0.78202, saving model to ./models/seed=2_04_val_acc:0.7820.hdf5
Epoch 5/500
37/37 [==============================] -

...

37/37 [==============================] - 3s 72ms/step - loss: 0.1310 - acc: 0.9672 - val_loss: 0.8929 - val_acc: 0.7439

Epoch 00022: val_acc did not improve from 0.77038
Epoch 23/500
37/37 [==============================] - 3s 73ms/step - loss: 0.1245 - acc: 0.9689 - val_loss: 0.9155 - val_acc: 0.7380

Epoch 00023: val_acc did not improve from 0.77038
Epoch 24/500
37/37 [==============================] - 3s 74ms/step - loss: 0.1263 - acc: 0.9642 - val_loss: 0.9107 - val_acc: 0.7387

Epoch 00024: val_acc did not improve from 0.77038
Epoch 25/500
37/37 [==============================] - 3s 74ms/step - loss: 0.1293 - acc: 0.9655 - val_loss: 1.0918 - val_acc: 0.7154

Epoch 00025: val_acc did not improve from 0.77038
Epoch 26/500
37/37 [==============================] - 3s 73ms/step - loss: 0.1838 - acc: 0.9359 - val_loss: 0.8453 - val_acc: 0.7380

Epoch 00026: val_acc did not improve from 0.77038
Epoch 27/500
37/37 [==============================] - 3s 74ms/step - loss: 0.1255 - acc: 0.96

Epoch 1/500
37/37 [==============================] - 7s 86ms/step - loss: 0.6730 - acc: 0.6080 - val_loss: 0.6758 - val_acc: 0.5964

Epoch 00001: val_acc improved from -inf to 0.59638, saving model to ./models/seed=6_01_val_acc:0.5964.hdf5
Epoch 2/500
37/37 [==============================] - 3s 74ms/step - loss: 0.6629 - acc: 0.6149 - val_loss: 0.6409 - val_acc: 0.5964

Epoch 00002: val_acc did not improve from 0.59638
Epoch 3/500
37/37 [==============================] - 3s 74ms/step - loss: 0.5602 - acc: 0.7141 - val_loss: 0.5347 - val_acc: 0.7251

Epoch 00003: val_acc improved from 0.59638 to 0.72510, saving model to ./models/seed=6_03_val_acc:0.7251.hdf5
Epoch 4/500
37/37 [==============================] - 3s 73ms/step - loss: 0.4679 - acc: 0.7912 - val_loss: 0.5141 - val_acc: 0.7607

Epoch 00004: val_acc improved from 0.72510 to 0.76067, saving model to ./models/seed=6_04_val_acc:0.7607.hdf5
Epoch 5/500
37/37 [==============================] - 3s 74ms/step - loss: 0.4150 - acc: 0.8


Epoch 00042: val_acc did not improve from 0.76455
Epoch 43/500
37/37 [==============================] - 3s 73ms/step - loss: 0.1235 - acc: 0.9622 - val_loss: 0.8188 - val_acc: 0.7342

Epoch 00043: val_acc did not improve from 0.76455
Epoch 44/500
37/37 [==============================] - 3s 73ms/step - loss: 0.0904 - acc: 0.9782 - val_loss: 0.8969 - val_acc: 0.7290

Epoch 00044: val_acc did not improve from 0.76455
Epoch 45/500
37/37 [==============================] - 3s 75ms/step - loss: 0.0823 - acc: 0.9793 - val_loss: 0.9476 - val_acc: 0.7290

Epoch 00045: val_acc did not improve from 0.76455
Epoch 46/500
37/37 [==============================] - 3s 74ms/step - loss: 0.0765 - acc: 0.9819 - val_loss: 0.9857 - val_acc: 0.7257

Epoch 00046: val_acc did not improve from 0.76455
Epoch 47/500
37/37 [==============================] - 3s 74ms/step - loss: 0.0716 - acc: 0.9825 - val_loss: 1.0179 - val_acc: 0.7206

Epoch 00047: val_acc did not improve from 0.76455
Epoch 48/500
37/37 [=========

37/37 [==============================] - 3s 72ms/step - loss: 0.0592 - acc: 0.9879 - val_loss: 1.1245 - val_acc: 0.7419

Epoch 00066: val_acc did not improve from 0.76843
Epoch 67/500
37/37 [==============================] - 3s 73ms/step - loss: 0.0574 - acc: 0.9888 - val_loss: 1.1559 - val_acc: 0.7335

Epoch 00067: val_acc did not improve from 0.76843
Epoch 68/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0646 - acc: 0.9862 - val_loss: 1.1068 - val_acc: 0.7413

Epoch 00068: val_acc did not improve from 0.76843
Epoch 69/500
37/37 [==============================] - 3s 73ms/step - loss: 0.0599 - acc: 0.9879 - val_loss: 1.1091 - val_acc: 0.7335

Epoch 00069: val_acc did not improve from 0.76843
Epoch 70/500
37/37 [==============================] - 3s 74ms/step - loss: 0.0651 - acc: 0.9851 - val_loss: 1.1314 - val_acc: 0.7303

Epoch 00070: val_acc did not improve from 0.76843
Epoch 71/500
37/37 [==============================] - 3s 74ms/step - loss: 0.0589 - acc: 0.98

37/37 [==============================] - 3s 74ms/step - loss: 0.5530 - acc: 0.7161 - val_loss: 0.5426 - val_acc: 0.7354

Epoch 00003: val_acc improved from 0.58991 to 0.73545, saving model to ./models/seed=9_03_val_acc:0.7354.hdf5
Epoch 4/500
37/37 [==============================] - 3s 73ms/step - loss: 0.4424 - acc: 0.8078 - val_loss: 0.5434 - val_acc: 0.7490

Epoch 00004: val_acc improved from 0.73545 to 0.74903, saving model to ./models/seed=9_04_val_acc:0.7490.hdf5
Epoch 5/500
37/37 [==============================] - 3s 75ms/step - loss: 0.3838 - acc: 0.8403 - val_loss: 0.6018 - val_acc: 0.7348

Epoch 00005: val_acc did not improve from 0.74903
Epoch 6/500
37/37 [==============================] - 3s 73ms/step - loss: 0.3586 - acc: 0.8520 - val_loss: 0.5732 - val_acc: 0.7542

Epoch 00006: val_acc improved from 0.74903 to 0.75420, saving model to ./models/seed=9_06_val_acc:0.7542.hdf5
Epoch 7/500
37/37 [==============================] - 3s 75ms/step - loss: 0.3308 - acc: 0.8652 - val

- 정확도 df 만들고 평균 확인 : [0.75789133]

In [48]:
df

,0
0,0.758085
1,0.772962
2,0.756792
3,0.751617
4,0.774903
5,0.774256
6,0.773609
7,0.733506
8,0.760673
9,0.722510


In [9]:
# RNN model
def rnn_model():
    seed_num = 42 # model_seed
    tf.random.set_seed(seed_num)

    rnn = Sequential()
    rnn.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
    rnn.add(SimpleRNN(units=128, activation='relu', return_sequences=True))
    rnn.add(SimpleRNN(units=64, activation='relu', return_sequences=True))
    rnn.add(SimpleRNN(units=32, activation='relu', return_sequences=False))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units=1))
    return rnn

def gru_model(X_train):
    seed_num = 42 # model_seed
    tf.random.set_seed(seed_num)

    gru = Sequential()
    gru.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
    gru.add(GRU(units=128, activation='relu', return_sequences=True))
    gru.add(GRU(units=64, activation='relu', return_sequences=True))
    gru.add(GRU(units=32, activation='relu', return_sequences=False))
    gru.add(Dropout(0.2))
    gru.add(Dense(units=1))
    return gru

def sorted_ls(path):
    mtime=lambda f: os.stat(os.path.join(path, f)).st_mtime
    return list(sorted(os.listdir(path), key=mtime))

In [10]:
def time_series_cv(name):
    path = '/project/LSH/'
    with tf.device('/device:GPU:0'):
        # 1. Data load
        x = np.load(path + 'x_(7727,10,4068).npy')
        y = np.load(path + 'y_(7727,1).npy')

        # 3-1. Best model saving
        MODEL_SAVE_FOLDER_PATH = './models/'
        if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
            os.mkdir(MODEL_SAVE_FOLDER_PATH)

        data={}
        sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)

        # 4. Crossvalidation
        for seed, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
            X_train, y_train = x[train_index,:,:], y[train_index]
            X_test, y_test = x[test_index,:,:], y[test_index]

            # 3-2. Best model saving
#             model_path = MODEL_SAVE_FOLDER_PATH + name +'{epoch:02d}_val_acc:{val_acc:.4f}.hdf5'

#             cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc', verbose=1, save_best_only=True)
            early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=False)

            if name == 'LSTM':
                model = lstm_model()
            elif name == 'RNN':
                model = rnn_model()
            elif name == 'GRU':
                model = gru_model(X_train)
                
            model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                          loss = "binary_crossentropy", metrics=['acc'])
            model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,
                      callbacks=[early_stop], shuffle=False)

#             5. accuracy checking 
            if name == 'LSTM':
                from pathlib import Path
                paths = sorted(Path(MODEL_SAVE_FOLDER_PATH).iterdir(), key=os.path.getmtime)[-1]
                best_model_path = str(paths)
                best_model_path = MODEL_SAVE_FOLDER_PATH + sorted_ls('models/')[-1]
                print('############# model path : ', best_model_path)

                best_model = keras.models.load_model(best_model_path) 
                y_pred_test = best_model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)
            else:    
                y_pred_test = model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)

            y_pred_test[y_pred_test>0.5]=1
            y_pred_test[y_pred_test<=0.5]=0

            acc = accuracy_score(y_test, y_pred_test)
            data[seed]=acc
            print(f'정확도 : {acc}, seed_num = {seed}')

        df = pd.DataFrame.from_dict(data, orient='index')
        print(f'정확도 df 만들고 평균 확인 : {df.mean().values}')    
    return df

In [74]:
rnn_df = time_series_cv('RNN')
rnn_df

0it [00:00, ?it/s]

Epoch 1/500
37/37 [==============================] - 3s 29ms/step - loss: 0.9977 - acc: 0.5847 - val_loss: 0.6504 - val_acc: 0.6151
Epoch 2/500
37/37 [==============================] - 1s 23ms/step - loss: 0.5978 - acc: 0.6926 - val_loss: 0.7343 - val_acc: 0.7076
Epoch 3/500
37/37 [==============================] - 1s 22ms/step - loss: 0.5079 - acc: 0.7750 - val_loss: 0.6607 - val_acc: 0.7160
Epoch 4/500
37/37 [==============================] - 1s 23ms/step - loss: 0.3940 - acc: 0.8388 - val_loss: 0.6509 - val_acc: 0.6475
Epoch 5/500
37/37 [==============================] - 1s 21ms/step - loss: 0.4368 - acc: 0.8255 - val_loss: 0.6925 - val_acc: 0.7141
Epoch 6/500
37/37 [==============================] - 1s 22ms/step - loss: 0.3378 - acc: 0.8630 - val_loss: 0.8493 - val_acc: 0.7212
Epoch 7/500
37/37 [==============================] - 1s 22ms/step - loss: 0.2459 - acc: 0.9143 - val_loss: 1.0385 - val_acc: 0.7044
Epoch 8/500
37/37 [==============================] - 1s 22ms/step - loss: 0.

37/37 [==============================] - 1s 20ms/step - loss: 0.0302 - acc: 0.9981 - val_loss: 3.3672 - val_acc: 0.7128
Epoch 63/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0299 - acc: 0.9981 - val_loss: 3.3384 - val_acc: 0.7115
Epoch 64/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0298 - acc: 0.9981 - val_loss: 3.3878 - val_acc: 0.7147
Epoch 65/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0298 - acc: 0.9981 - val_loss: 3.3866 - val_acc: 0.7135
Epoch 66/500
37/37 [==============================] - 1s 21ms/step - loss: 0.0300 - acc: 0.9981 - val_loss: 3.3592 - val_acc: 0.7135
Epoch 67/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0298 - acc: 0.9981 - val_loss: 3.3327 - val_acc: 0.7135
Epoch 68/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0298 - acc: 0.9981 - val_loss: 3.4395 - val_acc: 0.7180
Epoch 69/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0367 

37/37 [==============================] - 1s 23ms/step - loss: 0.0268 - acc: 0.9983 - val_loss: 3.3005 - val_acc: 0.7173
Epoch 46/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0270 - acc: 0.9983 - val_loss: 3.2890 - val_acc: 0.7167
Epoch 47/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0268 - acc: 0.9983 - val_loss: 3.3155 - val_acc: 0.7180
Epoch 48/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0268 - acc: 0.9983 - val_loss: 3.3156 - val_acc: 0.7167
Epoch 49/500
37/37 [==============================] - 1s 21ms/step - loss: 0.0271 - acc: 0.9981 - val_loss: 3.2636 - val_acc: 0.7225
Epoch 50/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0271 - acc: 0.9983 - val_loss: 3.2387 - val_acc: 0.7199
Epoch 51/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0266 - acc: 0.9983 - val_loss: 3.2551 - val_acc: 0.7251
Epoch 52/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0268 

37/37 [==============================] - 1s 21ms/step - loss: 0.0540 - acc: 0.9963 - val_loss: 3.2629 - val_acc: 0.7335
Epoch 53/500
37/37 [==============================] - 1s 21ms/step - loss: 0.0542 - acc: 0.9963 - val_loss: 3.1764 - val_acc: 0.7380
Epoch 54/500
37/37 [==============================] - 1s 21ms/step - loss: 0.0535 - acc: 0.9965 - val_loss: 3.2216 - val_acc: 0.7374
Epoch 55/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0536 - acc: 0.9965 - val_loss: 3.1567 - val_acc: 0.7367
Epoch 00055: early stopping
정확도 : 0.7270375161707633, seed_num = 2
Epoch 1/500
37/37 [==============================] - 2s 31ms/step - loss: 1.0760 - acc: 0.5864 - val_loss: 0.6852 - val_acc: 0.5854
Epoch 2/500
37/37 [==============================] - 1s 21ms/step - loss: 0.6104 - acc: 0.6906 - val_loss: 0.6519 - val_acc: 0.7180
Epoch 3/500
37/37 [==============================] - 1s 23ms/step - loss: 0.4672 - acc: 0.7940 - val_loss: 0.7687 - val_acc: 0.7497
Epoch 4/500
37/37 

...

37/37 [==============================] - 1s 23ms/step - loss: 0.0393 - acc: 0.9937 - val_loss: 2.1539 - val_acc: 0.7354
Epoch 16/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0355 - acc: 0.9937 - val_loss: 2.1984 - val_acc: 0.7367
Epoch 17/500
37/37 [==============================] - 1s 25ms/step - loss: 0.0357 - acc: 0.9935 - val_loss: 1.9631 - val_acc: 0.7283
Epoch 18/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0400 - acc: 0.9931 - val_loss: 2.2881 - val_acc: 0.7329
Epoch 19/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0354 - acc: 0.9955 - val_loss: 2.3659 - val_acc: 0.7348
Epoch 20/500
37/37 [==============================] - 1s 25ms/step - loss: 0.0302 - acc: 0.9968 - val_loss: 2.5504 - val_acc: 0.7322
Epoch 21/500
37/37 [==============================] - 1s 25ms/step - loss: 0.0277 - acc: 0.9968 - val_loss: 2.6001 - val_acc: 0.7380
Epoch 22/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0258 

37/37 [==============================] - 1s 24ms/step - loss: 0.0271 - acc: 0.9981 - val_loss: 3.2417 - val_acc: 0.7335
Epoch 77/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0267 - acc: 0.9983 - val_loss: 3.1907 - val_acc: 0.7380
Epoch 78/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0273 - acc: 0.9978 - val_loss: 3.2525 - val_acc: 0.7354
Epoch 79/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0266 - acc: 0.9983 - val_loss: 3.2947 - val_acc: 0.7361
Epoch 80/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0264 - acc: 0.9983 - val_loss: 3.2687 - val_acc: 0.7361
Epoch 81/500
37/37 [==============================] - 1s 25ms/step - loss: 0.0266 - acc: 0.9981 - val_loss: 3.3025 - val_acc: 0.7380
Epoch 82/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0269 - acc: 0.9981 - val_loss: 3.3253 - val_acc: 0.7322
Epoch 83/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0267 

37/37 [==============================] - 1s 22ms/step - loss: 0.0636 - acc: 0.9875 - val_loss: 2.5869 - val_acc: 0.7225
Epoch 48/500
37/37 [==============================] - 1s 22ms/step - loss: 0.2138 - acc: 0.9499 - val_loss: 2.1700 - val_acc: 0.7277
Epoch 49/500
37/37 [==============================] - 1s 24ms/step - loss: 0.3833 - acc: 0.9258 - val_loss: 1.8335 - val_acc: 0.7173
Epoch 50/500
37/37 [==============================] - 1s 22ms/step - loss: 0.1209 - acc: 0.9670 - val_loss: 2.0032 - val_acc: 0.7238
Epoch 51/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0626 - acc: 0.9875 - val_loss: 2.3218 - val_acc: 0.7219
Epoch 52/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0732 - acc: 0.9843 - val_loss: 2.4312 - val_acc: 0.7251
Epoch 53/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0482 - acc: 0.9899 - val_loss: 2.5759 - val_acc: 0.7257
Epoch 54/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0404 

37/37 [==============================] - 1s 25ms/step - loss: 0.0526 - acc: 0.9879 - val_loss: 2.4795 - val_acc: 0.7374
Epoch 21/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0485 - acc: 0.9873 - val_loss: 2.7353 - val_acc: 0.7419
Epoch 22/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0461 - acc: 0.9924 - val_loss: 2.6455 - val_acc: 0.7387
Epoch 23/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0401 - acc: 0.9935 - val_loss: 2.7374 - val_acc: 0.7374
Epoch 24/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0395 - acc: 0.9953 - val_loss: 2.7721 - val_acc: 0.7406
Epoch 25/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0360 - acc: 0.9944 - val_loss: 2.8964 - val_acc: 0.7432
Epoch 26/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0317 - acc: 0.9953 - val_loss: 2.8893 - val_acc: 0.7406
Epoch 27/500
37/37 [==============================] - 1s 24ms/step - loss: 0.1095 

,0
0,0.708926
1,0.728331
2,0.727038
3,0.731565
4,0.735446
5,0.717335
6,0.734799
7,0.737387
8,0.748383
9,0.714748


In [11]:
gru_df = time_series_cv('GRU')
gru_df

0it [00:00, ?it/s]

Epoch 1/500
37/37 [==============================] - 5s 56ms/step - loss: 1.0187 - acc: 0.5614 - val_loss: 0.6242 - val_acc: 0.6475
Epoch 2/500
37/37 [==============================] - 2s 44ms/step - loss: 0.5891 - acc: 0.6891 - val_loss: 0.6561 - val_acc: 0.7025
Epoch 3/500
37/37 [==============================] - 2s 45ms/step - loss: 0.5005 - acc: 0.7786 - val_loss: 0.7128 - val_acc: 0.7303
Epoch 4/500
37/37 [==============================] - 2s 44ms/step - loss: 0.4518 - acc: 0.8138 - val_loss: 0.7643 - val_acc: 0.6009
Epoch 5/500
37/37 [==============================] - 2s 44ms/step - loss: 0.5346 - acc: 0.7825 - val_loss: 0.9326 - val_acc: 0.7154
Epoch 6/500
37/37 [==============================] - 2s 43ms/step - loss: 0.3715 - acc: 0.8621 - val_loss: 1.1003 - val_acc: 0.7193
Epoch 7/500
37/37 [==============================] - 2s 45ms/step - loss: 0.2777 - acc: 0.9087 - val_loss: 1.8569 - val_acc: 0.7322
Epoch 8/500
37/37 [==============================] - 2s 44ms/step - loss: 0.

37/37 [==============================] - 2s 44ms/step - loss: 0.0573 - acc: 0.9957 - val_loss: 3.6153 - val_acc: 0.7354
Epoch 60/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0566 - acc: 0.9963 - val_loss: 3.7106 - val_acc: 0.7303
Epoch 61/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0570 - acc: 0.9963 - val_loss: 3.6866 - val_acc: 0.7296
Epoch 62/500
37/37 [==============================] - 2s 43ms/step - loss: 0.0572 - acc: 0.9959 - val_loss: 3.7260 - val_acc: 0.7322
Epoch 63/500
37/37 [==============================] - 2s 42ms/step - loss: 0.0599 - acc: 0.9961 - val_loss: 3.7199 - val_acc: 0.7354
Epoch 64/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0572 - acc: 0.9961 - val_loss: 3.7455 - val_acc: 0.7303
Epoch 00064: early stopping
정확도 : 0.7399741267787839, seed_num = 0
Epoch 1/500
37/37 [==============================] - 4s 54ms/step - loss: 1.1527 - acc: 0.5420 - val_loss: 0.6122 - val_acc: 0.6675
Epoch 2/500
37/3

37/37 [==============================] - 2s 44ms/step - loss: 0.0876 - acc: 0.9741 - val_loss: 2.0316 - val_acc: 0.7484
Epoch 53/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0626 - acc: 0.9836 - val_loss: 2.3106 - val_acc: 0.7445
Epoch 54/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0501 - acc: 0.9864 - val_loss: 2.5271 - val_acc: 0.7471
Epoch 55/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0426 - acc: 0.9877 - val_loss: 2.5673 - val_acc: 0.7426
Epoch 56/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0415 - acc: 0.9896 - val_loss: 2.6668 - val_acc: 0.7503
Epoch 57/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0355 - acc: 0.9914 - val_loss: 2.7243 - val_acc: 0.7445
Epoch 58/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0346 - acc: 0.9940 - val_loss: 2.8387 - val_acc: 0.7484
Epoch 59/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0314 

37/37 [==============================] - 2s 45ms/step - loss: 0.0717 - acc: 0.9894 - val_loss: 2.7396 - val_acc: 0.7432
Epoch 38/500
37/37 [==============================] - 2s 43ms/step - loss: 0.0669 - acc: 0.9920 - val_loss: 2.8940 - val_acc: 0.7413
Epoch 39/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0702 - acc: 0.9909 - val_loss: 2.7959 - val_acc: 0.7406
Epoch 40/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0639 - acc: 0.9933 - val_loss: 2.9324 - val_acc: 0.7400
Epoch 41/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0626 - acc: 0.9937 - val_loss: 3.0311 - val_acc: 0.7432
Epoch 42/500
37/37 [==============================] - 2s 43ms/step - loss: 0.0612 - acc: 0.9942 - val_loss: 3.0958 - val_acc: 0.7439
Epoch 43/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0606 - acc: 0.9948 - val_loss: 3.1437 - val_acc: 0.7490
Epoch 44/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0595 

...

37/37 [==============================] - 2s 43ms/step - loss: 0.0561 - acc: 0.9955 - val_loss: 3.3212 - val_acc: 0.7380
Epoch 36/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0547 - acc: 0.9961 - val_loss: 3.3466 - val_acc: 0.7387
Epoch 37/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0551 - acc: 0.9959 - val_loss: 3.4022 - val_acc: 0.7380
Epoch 38/500
37/37 [==============================] - 2s 43ms/step - loss: 0.0584 - acc: 0.9955 - val_loss: 3.4200 - val_acc: 0.7451
Epoch 39/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0543 - acc: 0.9963 - val_loss: 3.5506 - val_acc: 0.7380
Epoch 40/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0622 - acc: 0.9946 - val_loss: 3.3597 - val_acc: 0.7471
Epoch 41/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0604 - acc: 0.9946 - val_loss: 3.3104 - val_acc: 0.7529
Epoch 42/500
37/37 [==============================] - 2s 45ms/step - loss: 0.1483 

37/37 [==============================] - 2s 45ms/step - loss: 0.0573 - acc: 0.9957 - val_loss: 3.1853 - val_acc: 0.7646
Epoch 39/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0570 - acc: 0.9959 - val_loss: 3.1421 - val_acc: 0.7581
Epoch 40/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0574 - acc: 0.9957 - val_loss: 3.0962 - val_acc: 0.7600
Epoch 41/500
37/37 [==============================] - 2s 43ms/step - loss: 0.1654 - acc: 0.9845 - val_loss: 3.1494 - val_acc: 0.7451
Epoch 42/500
37/37 [==============================] - 2s 44ms/step - loss: 0.2413 - acc: 0.9728 - val_loss: 2.5735 - val_acc: 0.7413
Epoch 43/500
37/37 [==============================] - 2s 42ms/step - loss: 0.1443 - acc: 0.9802 - val_loss: 2.4816 - val_acc: 0.7490
Epoch 44/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0662 - acc: 0.9901 - val_loss: 2.6726 - val_acc: 0.7503
Epoch 45/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0579 

37/37 [==============================] - 2s 45ms/step - loss: 0.0273 - acc: 0.9955 - val_loss: 3.1734 - val_acc: 0.7380
Epoch 28/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0258 - acc: 0.9972 - val_loss: 3.2803 - val_acc: 0.7380
Epoch 29/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0248 - acc: 0.9972 - val_loss: 3.2626 - val_acc: 0.7361
Epoch 30/500
37/37 [==============================] - 2s 43ms/step - loss: 0.0262 - acc: 0.9961 - val_loss: 3.1705 - val_acc: 0.7439
Epoch 31/500
37/37 [==============================] - 2s 43ms/step - loss: 0.0282 - acc: 0.9965 - val_loss: 3.1453 - val_acc: 0.7451
Epoch 32/500
37/37 [==============================] - 2s 44ms/step - loss: 0.1336 - acc: 0.9814 - val_loss: 3.7794 - val_acc: 0.6831
Epoch 33/500
37/37 [==============================] - 2s 43ms/step - loss: 0.4777 - acc: 0.9172 - val_loss: 2.0867 - val_acc: 0.7458
Epoch 34/500
37/37 [==============================] - 2s 45ms/step - loss: 0.1211 

37/37 [==============================] - 2s 45ms/step - loss: 0.0308 - acc: 0.9976 - val_loss: 3.1641 - val_acc: 0.7354
Epoch 89/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0304 - acc: 0.9981 - val_loss: 3.1873 - val_acc: 0.7361
Epoch 90/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0302 - acc: 0.9981 - val_loss: 3.2103 - val_acc: 0.7316
Epoch 91/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0271 - acc: 0.9981 - val_loss: 3.1835 - val_acc: 0.7354
Epoch 92/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0268 - acc: 0.9983 - val_loss: 3.1834 - val_acc: 0.7348
Epoch 93/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0265 - acc: 0.9983 - val_loss: 3.2046 - val_acc: 0.7348
Epoch 94/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0267 - acc: 0.9983 - val_loss: 3.2325 - val_acc: 0.7354
Epoch 95/500
37/37 [==============================] - 2s 44ms/step - loss: 0.0265 

,0
0,0.739974
1,0.769728
2,0.737387
3,0.757439
4,0.754851
5,0.739974
6,0.747089
7,0.747736
8,0.752911
9,0.733506


# 비시계열

In [29]:
def non_time_series_cv(model):
    path = '/project/LSH/'
    with tf.device('/device:GPU:0'):
        # 1. Data load
        x = np.load(path + 'x_(7727,4068).npy')
        y = np.load(path + 'y_(7727,1).npy')

        # 3-1. Best model saving
        MODEL_SAVE_FOLDER_PATH = './models/'
        if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
            os.mkdir(MODEL_SAVE_FOLDER_PATH)

        acc_list = []
        sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)

        # 4. Crossvalidation
        for seed, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
            X_train, y_train = x[train_index,:], y[train_index]
            X_test, y_test = x[test_index,:], y[test_index]
            model.fit(X_train, y_train)
            pred = model.predict(X_test)
            acc = accuracy_score(y_test, pred)
            print(f'{str(model)} : {acc}')
            acc_list.append(acc)
        mean_pred = np.mean(acc_list)
        print(f'{str(model)} MEAN : {mean_pred}')
        return mean_pred

In [30]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

result = []

# Support vector machine
model = SVC()
result.append({'model' : str(model), 'mean' : non_time_series_cv(model)})

# Gaussian naive bayes
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
result.append({'model' : str(model), 'mean' : non_time_series_cv(model)})

# K nearest neighbor
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
result.append({'model' : str(model), 'mean' : non_time_series_cv(model)})

# Random Forest
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
result.append({'model' : str(model), 'mean' : non_time_series_cv(model)})

# Gradient boosting
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
result.append({'model' : str(model), 'mean' : non_time_series_cv(model)})

# Neural network
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(max_iter=1000)
result.append({'model' : str(model), 'mean' : non_time_series_cv(model)})

# AdaBoost
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=500, 
                        random_state=10, 
                        learning_rate=0.1)
result.append({'model' : str(model), 'mean' : non_time_series_cv(model)})

0it [00:00, ?it/s]

SVC() : 0.779430789133247
SVC() : 0.7807244501940491
SVC() : 0.7619663648124192
SVC() : 0.7729624838292367
SVC() : 0.7613195342820182
SVC() : 0.7723156532988357
SVC() : 0.7736093143596378
SVC() : 0.7639068564036222
SVC() : 0.7749029754204398
SVC() : 0.7729624838292367
SVC() MEAN : 0.7714100905562742


0it [00:00, ?it/s]

GaussianNB() : 0.48641655886157825
GaussianNB() : 0.4689521345407503
GaussianNB() : 0.4987063389391979
GaussianNB() : 0.47347994825355755
GaussianNB() : 0.48124191461837
GaussianNB() : 0.4922380336351876
GaussianNB() : 0.49029754204398446
GaussianNB() : 0.47477360931435963
GaussianNB() : 0.4831824062095731
GaussianNB() : 0.4870633893919793
GaussianNB() MEAN : 0.48363518758085383


0it [00:00, ?it/s]

...

GradientBoostingClassifier() : 0.7561448900388098
GradientBoostingClassifier() : 0.7496765847347995
GradientBoostingClassifier() : 0.7380336351875808
GradientBoostingClassifier() : 0.740620957309185
GradientBoostingClassifier() : 0.7445019404915912
GradientBoostingClassifier() : 0.7496765847347995
GradientBoostingClassifier() : 0.7613195342820182
GradientBoostingClassifier() : 0.741267787839586
GradientBoostingClassifier() : 0.7432082794307892
GradientBoostingClassifier() : 0.7600258732212161
GradientBoostingClassifier() MEAN : 0.7484476067270374


0it [00:00, ?it/s]

MLPClassifier(max_iter=1000) : 0.7289780077619664
MLPClassifier(max_iter=1000) : 0.7438551099611902
MLPClassifier(max_iter=1000) : 0.740620957309185
MLPClassifier(max_iter=1000) : 0.7445019404915912
MLPClassifier(max_iter=1000) : 0.7153945666235446
MLPClassifier(max_iter=1000) : 0.7212160413971539
MLPClassifier(max_iter=1000) : 0.7373868046571799
MLPClassifier(max_iter=1000) : 0.7302716688227684
MLPClassifier(max_iter=1000) : 0.7289780077619664
MLPClassifier(max_iter=1000) : 0.7335058214747736
MLPClassifier(max_iter=1000) MEAN : 0.732470892626132


0it [00:00, ?it/s]

AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7548512289780077
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7542043984476067
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7457956015523933
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7529107373868047
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7483829236739974
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.758732212160414
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7690815006468306
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7399741267787839
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7399741267787839
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7580853816300129
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, rando

In [35]:
result.append({'model' : 'LSTM', 'mean' : 0.75789133})
df = pd.DataFrame(result)
df

,model,mean
0,SVC(),0.771410
1,GaussianNB(),0.483635
2,KNeighborsClassifier(),0.659056
3,RandomForestClassifier(),0.738422
4,GradientBoostingClassifier(),0.748448
5,MLPClassifier(max_iter=1000),0.732471
6,"AdaBoostClassifier(learning_rate=0.1, n_estima...",0.752199
7,LSTM,0.757891


In [36]:
df.sort_values('mean')

,model,mean
1,GaussianNB(),0.483635
2,KNeighborsClassifier(),0.659056
5,MLPClassifier(max_iter=1000),0.732471
3,RandomForestClassifier(),0.738422
4,GradientBoostingClassifier(),0.748448
6,"AdaBoostClassifier(learning_rate=0.1, n_estima...",0.752199
7,LSTM,0.757891
0,SVC(),0.771410
